<center>
<br />
<h1>Ускорение кода, Numpy, cython</h1>
<h3>Python</h3>
<br />
<h4>2019</h4> </center>

# Matrix

Создадим класс Matrix:

In [8]:
import random

class Matrix(list):
    @classmethod
    def zeros(cls, shape):
        return cls([[0] * shape[1] for i in range(shape[0])])

    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255) for j in range(n_cols)])
        return M
    
    def transpose(self):
        n_rows, n_cols = self.shape
        return self.__class__(zip(*self))

    @property
    def shape(self):
        return (0, 0) if not self else (len(self), len(self[0]))

Кубическая самая простая реализация умножения:

In [2]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    
    assert n_xcols == n_yrows, "Incompatible matrix dimensions"
    
    Z = Matrix.zeros((n_xrows, n_ycols))
    
    for i in range(n_xrows):
        for j in range(n_xcols):
            for k in range(n_ycols):
                Z[i][k] += X[i][j] * Y[j][k]
    return Z

 

Проверим, что работает корректно:

In [3]:
X = Matrix([[1], [2], [3]])
Y = Matrix([[4, 5, 6]])
print(matrix_product(X, Y))
print(matrix_product(Y, X))


[[4, 5, 6], [8, 10, 12], [12, 15, 18]]
[[32]]


Посмотрим, сколько будет работать такое умножение:

In [4]:
def setup(shape=(100, 100)):
    X = Matrix.random(shape)
    Y = Matrix.random(shape)
    return X, Y

In [5]:
shape = (100, 100)

X = Matrix.random(shape)
Y = Matrix.random(shape)

In [6]:
%%timeit
matrix_product(*setup())

513 ms ± 29.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Ого! Пол секунды - непозволительно долго. <br>
Посмотрим, где именно мы теряем время, для этого можно установить line_profiler

In [7]:
#!pip3 install --user line_profiler

In [7]:
%load_ext line_profiler

In [8]:
%lprun -f matrix_product matrix_product(*setup())

```
Timer unit: 1e-06 s

Total time: 1.35886 s
File: <ipython-input-2-b9cb39bb9b1b>
Function: matrix_product at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def matrix_product(X, Y):
     2         1         20.0     20.0      0.0      n_xrows, n_xcols = X.shape
     3         1          9.0      9.0      0.0      n_yrows, n_ycols = Y.shape
     4                                               
     5         1          3.0      3.0      0.0      assert n_xcols == n_yrows, "Incompatible matrix dimensions"
     6                                               
     7         1        274.0    274.0      0.0      Z = Matrix.zeros((n_xrows, n_ycols))
     8                                               
     9       101         37.0      0.4      0.0      for i in range(n_xrows):
    10     10100       3939.0      0.4      0.3          for j in range(n_xcols):
    11   1010000     392582.0      0.4     28.9              for k in range(n_ycols):
    12   1000000     961999.0      1.0     70.8                  Z[i][k] += X[i][j] * Y[j][k]
    13         1          1.0      1.0      0.0      return Z
```

Видим, что куча времени уходит на for и на доставание индексов. Как думаете, что можно улучшить?

In [9]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        for j in range(n_xcols):
            for k in range(n_ycols):
                Z[i][k] += X[i][j] * Y[j][k]
    return Z

In [10]:
%%timeit
matrix_product(*setup())

456 ms ± 8.72 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Попробуем убрать лишнее доставание элемента по индексу:

In [11]:
def matrix_product2(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        Xi = X[i]
        for k in range(n_ycols):
            acc = 0
            for j in range(n_xcols):
                acc += Xi[j] * Y[j][k]
            Z[i][k] = acc
    return Z

In [12]:
%%timeit
matrix_product2(*setup())

241 ms ± 8.79 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Стало в 2 раза быстрее! Так как стало примерно в 2 раза меньше операций доставания по индексу.

In [13]:
%lprun -f matrix_product2 matrix_product2(*setup())

```
Timer unit: 1e-06 s

Total time: 1.03374 s
File: <ipython-input-14-e24c1e2d67bf>
Function: matrix_product2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def matrix_product2(X, Y):
     2         1          6.0      6.0      0.0      n_xrows, n_xcols = X.shape
     3         1          1.0      1.0      0.0      n_yrows, n_ycols = Y.shape
     4         1         42.0     42.0      0.0      Z = Matrix.zeros((n_xrows, n_ycols))
     5       101         37.0      0.4      0.0      for i in range(n_xrows):
     6       100         59.0      0.6      0.0          Xi = X[i]
     7     10100       4020.0      0.4      0.4          for k in range(n_ycols):
     8     10000       3606.0      0.4      0.3              acc = 0
     9   1010000     388146.0      0.4     37.5              for j in range(n_xcols):
    10   1000000     631985.0      0.6     61.1                  acc += Xi[j] * Y[j][k]
    11     10000       5840.0      0.6      0.6              Z[i][k] = acc
    12         1          1.0      1.0      0.0      return Z
```

Пойдём дальше и попробуем ещё ускориться

In [14]:
def matrix_product3(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose()  # :)
    for i, (Xi, Zi) in enumerate(zip(X, Z)):
        for k, Ytk in enumerate(Yt):
            getX = Xi.__getitem__
            Zi[k] = sum(getX(j) * Ytk[j] for j in range(n_xcols))
    return Z

In [15]:
%%timeit
matrix_product3(*setup())

145 ms ± 1.71 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Стало ещё почти в 2 раза быстрее!

Вывод: если ваш код тормозит, не спешите использовать другие фреймворки. Часто можно ускорить в разы свой код небольшими манипуляциями.

# Numpy

Попробуем взять нашу первую самую тупую реализацию, но использовать не наш класс Matrix, а реализацию массива из numpy (позже посмотрим, как она устроена)

In [16]:
import numpy as np

def np_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [17]:
def setup_np(shape=(100, 100)):
    X = np.random.randint(-255, 255, size=shape)
    Y = np.random.randint(-255, 255, size=shape)
    return X, Y

In [18]:
%timeit np_matrix_product(*setup_np())

523 ms ± 41.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Никакого профита пока нет.

In [19]:
X, Y = setup_np()

In [21]:
%timeit np.dot(X, Y)

760 µs ± 18 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


А вот это уже интересно. Достойная мотивация для изучения модуля numpy.

## Array

In [24]:
a = np.array([4, 1.1, 2, 3])

a[1] = 5

print(a.size, a.shape, a.dtype)
print(a)

4 (4,) float64
[4. 5. 2. 3.]


Можно увидеть, что массив эффективно лежит в памяти:

In [25]:
a.nbytes

32

Можно менять shape:

In [26]:
a.shape = (2, 2)
a

array([[4., 5.],
       [2., 3.]])

Массив имеет один тип. Если при создании передать разные типы, то он всё равно всё приведёт к одному типу.

In [27]:
np.array([4, 1.1, 2, 3, "string"])

array(['4', '1.1', '2', '3', 'string'], dtype='<U32')

## UFuncs

UFunc (universal functions) - это функции, которые работают над массивом поэлементно.

In [28]:
a = range(4)
b = [value + 1 for value in a]
print(b)

[1, 2, 3, 4]


Такой код будет в numpy выглядеть так:

In [29]:
a = np.arange(4)
b = a + 1
print(b)

[1 2 3 4]


Это работает для любых универсальных функций

Сложение двух array происходит поэлементно и при этом быстро:

In [30]:
count = int(1e7)
a, b = [random.random() for _ in range(count)], [random.random() for _ in range(count)]

%timeit [i + j for (i, j) in zip(a, b)]

710 ms ± 7.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
a, b = np.array(a), np.array(b)
%timeit a + b

36.6 ms ± 208 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Это работает быстро, так как всё итерирование протащено в компилируемую часть кода.

Есть и другие UFunc, т.е. поэлементные функции. Например посмотрим, сколько займёт поэлементное возведение массива в квадрат:

In [32]:
def slow():
    a = range(10000)
    return [i ** 2 for i in a]

%timeit slow()

2.47 ms ± 134 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [33]:
def fast():
    a = np.arange(10000)
    return a ** 2

%timeit fast()

12.4 µs ± 149 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Есть много других UFuncs: арифметические операция, побитовые, сравнения, тригонометрические и т.д.

## Aggregation

В numpy много встроенных агрегационных функций

<img src="aggregations.png" width=600px>

In [34]:
data = [random.random() for _ in range(1000000)]
%timeit min(data)

22.1 ms ± 904 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [35]:
np_data = np.array(data)
%timeit np.min(np_data)

514 µs ± 12 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [36]:
X = np.random.randint(-255, 255, size=shape)
Y = np.random.randint(-255, 255, size=shape)

%timeit np.dot(X, Y)

800 µs ± 35.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Indexing

Есть все базовые слайсы и индексы. Но есть не мало дополнительных приятных особенностей:

In [37]:
X = np.arange(6).reshape((2, 3))
X

array([[0, 1, 2],
       [3, 4, 5]])

In [38]:
X[1]

array([3, 4, 5])

In [39]:
X[0, 1]

1

In [40]:
z = X[:1, 1:]
z

array([[1, 2]])

В отличае от обычных списков при слайсах не происходит копирования и можно изменить исходный список.

In [41]:
z[0, 0] = 100

In [42]:
X

array([[  0, 100,   2],
       [  3,   4,   5]])

In [43]:
b = np.arange(20)
b[:10] + b[10:]

array([10, 12, 14, 16, 18, 20, 22, 24, 26, 28])

### Masking

In [44]:
a = np.arange(8) ** 2
a[a > 8]

array([ 9, 16, 25, 36, 49])

Это работает за счёт того, что numpy.array поддерживает индексацию по булевой маске:

In [45]:
a > 8

array([False, False, False,  True,  True,  True,  True,  True])

Так же можно использовать массив индексов:

In [46]:
a = np.arange(4) * 2
print(a)
a[[3, 3, 2, 2, 0, 0, 1, 1]]

[0 2 4 6]


array([6, 6, 4, 4, 0, 0, 2, 2])

## Broadcasting
Или автоматическое приведение размерностей.

Можно складывать массивы разных размеров там, где это имеет смысл:

In [47]:
a = np.arange(3)
a + 100

array([100, 101, 102])

In [48]:
b = np.array(((0, 0, 0), (10, 10, 10), (20, 20, 20), (30, 30, 30)))
b

array([[ 0,  0,  0],
       [10, 10, 10],
       [20, 20, 20],
       [30, 30, 30]])

In [49]:
b + a

array([[ 0,  1,  2],
       [10, 11, 12],
       [20, 21, 22],
       [30, 31, 32]])

<img src="https://i.stack.imgur.com/JcKv1.png" width=900px/>

Но не всегда всё работает так, как хотелось бы:

In [50]:
X = np.arange(6).reshape((3, 2))
X

array([[0, 1],
       [2, 3],
       [4, 5]])

In [51]:
X + np.array([1, 2, 3])

ValueError: operands could not be broadcast together with shapes (3,2) (3,) 

Мы, конечно, понимаем, что имелось в виду, но numpy не понял. Подскажем ему:

In [52]:
Y = np.array((1, 2, 3))[:, np.newaxis]  # переориентируем массив
print(Y.shape)
print(Y)

(3, 1)
[[1]
 [2]
 [3]]


In [53]:
X + Y

array([[1, 2],
       [4, 5],
       [7, 8]])

Вывод. Циклы в python медленные. Стоит использовать встроенные функции numpy. Так и код понятнее и ускорение значительное.

# Cython 
Ещё один способ ускорить ваш код

Компилятор Cython переводит программу, написанную на языках Python или Cython, в более эффективный код на С

Язык Cython является надстройкой Python,
поддерживающей вызовы функций C и присвоение
переменным типов данных языка C.

Посмотрим, как он работает. Создадим пару функций на чистом python:

In [54]:
def f(x):
    return x**2

def integrate_f(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

In [55]:
%%timeit
integrate_f(0, 10, 1000000)

276 ms ± 11.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Теперь подключим Cython магию:

In [56]:
%load_ext Cython

In [57]:
%%cython -a 
def f(x):
    return x**2

def integrate_f2(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

Cython пытается определить тип и в частности за счёт этого ускориться. Но желтым подсвечиваются те строки кода, где он не смог установить тип. Тут почти всё желтое.

In [58]:
%%timeit
integrate_f2(0, 10, 1000000)

217 ms ± 7.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Стало чуть быстрее. Но можно лучше.

Применим статическую типизацию к переменным:

In [59]:
%%cython -a 
def f(double x):
    return x**2

def integrate_f3(double a, double b, int N):
    cdef int i
    cdef double s, dx
    s = 0
    dx = (b - a) / N
    for i in range(N):  # в случае, когда i объевлено как int, этот цикл компилируется в чистый C код
        s += f(a + i * dx)
    return s * dx

In [60]:
%%timeit
integrate_f3(0, 10, 1000000)

50.9 ms ± 2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Вызов функций в python может быть дорогим. Например, в
прошлом примере постоянно происходила конвертация из C
double в Python float. Изменим прототип функции f:

In [61]:
%%cython -a 
cdef double f2(double x):
    return x**2

def integrate_f4(double a, double b, int N):
    cdef int i
    cdef double s, dx
    s = 0
    dx = (b - a) / N
    for i in range(N):  # в случае, когда i объевлено как int, этот цикл компилируется в чистый C код
        s += f2(a + i * dx)
    return s * dx

In [62]:
%%timeit
integrate_f4(0, 10, 1000000)

1.27 ms ± 57.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Посмотрим, как справляется с той же задачей numpy.

In [63]:
%%timeit
x = np.linspace(0, 10, 1000000)
(x**2).sum() / 100000

7.36 ms ± 183 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Таким образом, с помощью Cython можно написать более эффективный код по сравнению с numpy!

## Matrix
Вернёмся к нашему примеру с Matrix

In [64]:
%%cython -a 
import random

class Matrix(list):
    @classmethod
    def zeros(cls, shape):
        n_rows, n_cols = shape
        return cls([[0] * n_cols for i in range(n_rows)])

    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255)
                      for j in range(n_cols)])
        return M

    def transpose(self):
        n_rows, n_cols = self.shape
        return self.__class__(zip(*self))

    @property
    def shape(self):
        return ((0, 0) if not self else
                (int(len(self)), int(len(self[0]))))


def cy_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose()
    for i, Xi in enumerate(X):
        for k, Ytk in enumerate(Yt):
            Z[i][k] = sum(Xi[j] * Ytk[j] for j in range(n_xcols))
    return Z


Пока всё желтое, т.е. cython не может распознать типы и обращается к интерпрератору python

In [65]:
shape

(100, 100)

In [66]:
X = Matrix.random(shape)
Y = Matrix.random(shape)

In [67]:
%timeit cy_matrix_product(X, Y)

86.9 ms ± 6.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Получилось быстрее, чем наша питоновская реализация, но пока сильно не дотягивает до numpy версии.<br>
Попробуем подсказать cython что за типы мы используем, вдруг так будет быстрее.

In [69]:
X = np.random.randint(-255, 255, size=shape, dtype=np.int64)
Y = np.random.randint(-255, 255, size=shape, dtype=np.int64)

In [70]:
%%cython -a
import numpy as np

def cy_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [71]:
%timeit cy_matrix_product(X, Y)

419 ms ± 17.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Не помогло, всё стало ещё медленнее.<br>
Дело в том, что cython не умеет работать с питонячими импортами и теперь все строки для него непонятны и он обращается к интерпретатору питона.

Окей, давайте явно подскажут cython'у про типы и про импорты.<br>
Используем cdef, чтобы определить переменные СИшных типов

In [72]:
%%cython -a
import numpy as np
cimport numpy as np

def cy_matrix_product(np.ndarray X, np.ndarray Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray Z
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [73]:
%timeit  cy_matrix_product(X, Y)

445 ms ± 26.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Вроде cython теперь больше понимает, но всё равно всё медленно, так как основные части всё ещё не понятны.

Добавим строгой типизации входных значений - теперь наша функция принимают только массивы определённого типа.

In [74]:
%%cython -a
import numpy as np
cimport numpy as np  # Не любой модуль можно так импортировать, а только те, которые это поодерживают


cdef cy_matrix_product(np.ndarray[np.int64_t, ndim=2] X,
                      np.ndarray[np.int64_t, ndim=2] Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray[np.int64_t, ndim=2] Z = \
        np.zeros((n_xrows, n_ycols), dtype=np.int64)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [75]:
%timeit -n10 cy_matrix_product(X, Y)

442 ms ± 23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Ну теперь добавим совсем уже чёрной магии.<br>
Добавим декораторы, которые проверяют границы массивов и переполнение типов.

In [76]:
%%cython 
import numpy as np

cimport cython
cimport numpy as np

@cython.boundscheck(False)
@cython.overflowcheck(False)
def cy_matrix_product(np.ndarray[np.int64_t, ndim=2] X, 
                      np.ndarray[np.int64_t, ndim=2] Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray[np.int64_t, ndim=2] Z = \
        np.zeros((n_xrows, n_ycols), dtype=np.int64)
    for i in range(n_xrows):        
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [77]:
%timeit cy_matrix_product(X, Y)

1.44 ms ± 11.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Profit.